Great work! The desk now has the price data they need. The final ingredient before they can begin trading with the client is the pricing model. Alex tells you the client wants to start trading as soon as possible. They believe the winter will be colder than expected, so they want to buy gas now to store and sell in winter in order to take advantage of the resulting increase in gas prices. They ask you to write a script that they can use to price the contract. Once the desk are happy, you will work with engineering, risk, and model validation to incorporate this model into production code.

The concept is simple: any trade agreement is as valuable as the price you can sell minus the price at which you are able to buy. Any cost incurred as part of executing this agreement is also deducted from the overall value. So, for example, if I can purchase a million MMBtu of natural gas in summer at $2/MMBtu, store this for four months, and ensure that I can sell the same quantity at $3/MMBtu without incurring any additional costs, the value of this contract would be ($3-$2) *1e6 = $1million. If there are costs involved, such as having to pay the storage facility owner a fixed fee of $100K a month, then the 'value' of the contract, from my perspective, would drop by the overall rental amount to $600K. Another cost could be the injection/withdrawal cost, like having to pay the storage facility owner $10K per 1 million MMBtu for injection/withdrawal, then the price will further go down by $10K to $590K. Additionally, if I am supposed to foot a bill of $50K each time for transporting the gas to and from the facility, the cost of this contract would fall by another $100K. Think of the valuation as a fair estimate at which both the trading desk and the client would be happy to enter into the contract. 

In [6]:
import pandas as pd
from datetime import datetime

# Load the data from a CSV file

prices_df = pd.read_csv('Nat_Gas.csv')
# Convert the 'Dates' column to datetime objects
prices_df['Dates'] = pd.to_datetime(prices_df['Dates'])
# Create a dictionary with dates as keys and prices as values
prices_dict = dict(zip(prices_df['Dates'], prices_df['Prices']))
 
# Calculate the net value of a gas storage contract
def calculate_contract_value(injection_dates, withdrawal_dates, purchase_prices, sale_prices, injection_costs, withdrawal_costs, transport_costs, storage_fee_per_month, max_storage_volume):
    """
    Calculate the net value of a gas storage contract.

    Parameters:
    - injection_dates: List of dates when gas is injected.
    - withdrawal_dates: List of dates when gas is withdrawn.
    - purchase_prices: Dictionary mapping dates to purchase prices per MMBtu.
    - sale_prices: Dictionary mapping dates to sale prices per MMBtu.
    - injection_costs: Cost to inject gas per MMBtu.
    - withdrawal_costs: Cost to withdraw gas per MMBtu.
    - transport_costs: Transportation cost per round trip.
    - storage_fee_per_month: Monthly storage fee.
    - max_storage_volume: Maximum volume of gas that can be stored.

    Returns:
    - float: Net value of the contract.
    """
    # Initialize variables
    total_cost = 0
    total_revenue = 0
    total_volume = 0

    # Calculate costs and revenues for each injection and corresponding withdrawal
    for inject_date, withdraw_date in zip(injection_dates, withdrawal_dates):
        if inject_date in purchase_prices and withdraw_date in sale_prices:
            volume = min(max_storage_volume, max_storage_volume)  # Simplified volume handling
            purchase_price = purchase_prices[inject_date]
            sale_price = sale_prices[withdraw_date]
            
            # Calculate revenues
            revenue = volume * (sale_price - purchase_price)
            total_revenue += revenue
            
            # Calculate costs
            inject_cost = volume * injection_costs
            withdraw_cost = volume * withdrawal_costs
            transport_cost = transport_costs
            storage_months = (withdraw_date - inject_date).days // 30  # Approximate month count
            storage_cost = storage_months * storage_fee_per_month
            
            total_cost += inject_cost + withdraw_cost + transport_cost + storage_cost

    # Net value calculation
    net_value = total_revenue - total_cost
    return net_value
# Example usage
csv_filepath = 'Nat_Gas.csv'  # Update this to your actual CSV file path


prices_df = pd.read_csv('Nat_Gas.csv')
# Convert the 'Dates' column to datetime objects
prices_df['Dates'] = pd.to_datetime(prices_df['Dates'])
# Create a dictionary with dates as keys and prices as values
prices_dict = dict(zip(prices_df['Dates'], prices_df['Prices']))
 
    
# Now we have to make sure that the dates are in datetime format
injection_dates = [datetime(2024, 6, 1), datetime(2024, 7, 1)]
withdrawal_dates = [datetime(2024, 10, 1), datetime(2024, 11, 1)]
# ... existing code for the rest of the parameters ...
injection_costs = 0.01
withdrawal_costs = 0.01
transport_costs = 50000
storage_fee_per_month = 100000
max_storage_volume = 1000000 

value = calculate_contract_value(injection_dates, withdrawal_dates, prices_dict, injection_costs, withdrawal_costs, transport_costs, storage_fee_per_month, max_storage_volume)
print(f"The net value of the contract is ${value:.2f}")


TypeError: calculate_contract_value() missing 1 required positional argument: 'max_storage_volume'